<a href="https://colab.research.google.com/github/lanehale/airline-chatbot/blob/main/pytorch05_ex.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from pathlib import Path

# Create the 'going_modular' directory if it doesn't exist
Path("going_modular").mkdir(exist_ok=True)

In [ ]:
# get_data.py

%%writefile going_modular/get_data.py
"""
Contains functionality for creating data folders and
downloading data.
"""
import os
import requests
import zipfile
from pathlib import Path

# Set up path to data folder
data_path = Path("data/")
image_path = data_path / "pizza_steak_sushi"

# If the image folder doesn't exist, download it and prepare it...
if image_path.is_dir():
  print(f"{image_path} directory exists.")
else:
  print(f"Did not find {image_path} directory, creating one...")
  image_path.mkdir(parents=True, exist_ok=True)

  # Download images
  with open(data_path / "pizza_steak_sushi_20_percent.zip", "wb") as f:
    request = requests.get("https://github.com/mrdbourke/pytorch-deep-learning/raw/main/data/pizza_steak_sushi_20_percent.zip")
    print("Downloading pizza, steak, sushi data...")
    f.write(request.content)

  # Unzip image data
  with zipfile.ZipFile(data_path / "pizza_steak_sushi_20_percent.zip", "r") as zip_ref:
    print("Unzipping pizza, steak, sushi data...")
    zip_ref.extractall(image_path)

  # Remove zip file
  os.remove(data_path / "pizza_steak_sushi_20_percent.zip")

In [ ]:
# Get images
!python going_modular/get_data.py

In [ ]:
# data_setup.py

%%writefile going_modular/data_setup.py
"""
Contains functionality for creating PyTorch DataLoaders for
image classification data.
"""
import os

from torchvision import datasets, transforms
from torch.utils.data import DataLoader

NUM_WORKERS = os.cpu_count()

def create_dataloaders(
    train_dir: str,
    test_dir: str,
    transform: transforms.Compose,
    batch_size: int,
    num_workers: int=NUM_WORKERS
):
  """Creates training and testing DataLoaders.

  Takes in a training directory and testing directory path and turns
  them into PyTorch Datasets and then into PyTorch DataLoaders.

  Args:
    train_dir: Path to training directory.
    test_dir: Path to testing directory.
    transform: torchvision transforms to perform on training and testing data.
    batch_size: Number of samples per batch in each of the DataLoaders.
    num_workers: An integer for number of workers per DataLoader.

  Returns:
    A tuple of (train_dataloader, test_dataloader, class_names).
    Where class_names is a list of the target classes.
    Example usage:
      train_dataloader, test_dataloader, class_names = \
        = create_dataloaders(train_dir=path/to/train_dir,
                             test_dir=path/to/test_dir,
                             transform=some_transform,
                             batch_size=32,
                             num_workers=4)
  """
  # Use ImageFolder to create datasets:
  train_data = datasets.ImageFolder(train_dir, transform=transform,
                                    target_transform=None)
  test_data = datasets.ImageFolder(test_dir, transform=transform)

  # Get class names
  class_names = train_data.classes

  # Turn images into data loaders
  train_dataloader = DataLoader(
      train_data,
      batch_size=batch_size,
      shuffle=True,
      num_workers=num_workers,
      pin_memory=True,
  )
  test_dataloader = DataLoader(
      test_data,
      batch_size=batch_size,
      shuffle=False,  # don't need to shuffle test data
      num_workers=num_workers,
      pin_memory=True,
  )

  return train_dataloader, test_dataloader, class_names

In [ ]:
!ls

In [ ]:
image_path = Path("data/pizza_steak_sushi")
train_data_path = image_path / "train"
test_data_path = image_path / "test"

from torchvision import transforms
data_transform_flip = transforms.Compose([
    transforms.Resize((224,224)),#64,64)),
    # Flip images randomly on the horizontal
    transforms.RandomHorizontalFlip(p=0.5),  # p = probability of flip, 0.5 = 50% chance
    transforms.ToTensor()
])

# If we'd like to make DataLoader's we can now use the function within data_setup.py like so:

from going_modular import data_setup  # Import data_setup.py

# Create train/test dataloader and get class names as a list
train_dataloader, test_dataloader, class_names = data_setup.create_dataloaders(
    train_data_path,
    test_data_path,
    data_transform_flip,
    batch_size=1
)
#    num_workers=2)  # os.cpu_count(), for number of workers per DataLoader
train_dataloader, test_dataloader, class_names

In [ ]:
# model_builder.py

%%writefile going_modular/model_builder.py
"""
Contains PyTorch model code to instantiate a TinyVGG model.
"""
import torch
from torch import nn

class TinyVGG(nn.Module):
  """Creates the TinyVGG architecture.

  Replicates the TinyVGG architecture from the CNN explainer website in PyTorch.
  See the original architecture here: https://poloclub.github.io/cnn-explainer/

  Args:
    input_shape: An integer indicating number of input channels.
    hidden_units: An integer indicating number of hidden units between layers.
    output_shape: An integer indicating number of output units.
  """
  def __init__(self, input_shape: int, hidden_units: int, output_shape: int) -> None:
    super().__init__()
    self.conv_block_1 = nn.Sequential(
        nn.Conv2d(in_channels=input_shape,
                  out_channels=hidden_units,
                  kernel_size=3,
                  stride=1,
                  padding=1),
        nn.ReLU(),
        nn.Conv2d(in_channels=hidden_units,
                  out_channels=hidden_units,
                  kernel_size=3,
                  stride=1,
                  padding=1),
        nn.ReLU(),
        nn.MaxPool2d(kernel_size=2,
                     stride=2)
    )
    self.conv_block_2 = nn.Sequential(
        nn.Conv2d(hidden_units, hidden_units, kernel_size=3, padding=1),
        nn.ReLU(),
        nn.Conv2d(hidden_units, hidden_units, kernel_size=3, padding=1),
        nn.ReLU(),
        nn.MaxPool2d(2)
    )
    self.classifier = nn.Sequential(
        nn.Flatten(),
        # Where did this in_features shape come from?
        # It's because each layer of our network compresses and changes the shape of our inputs data.
        nn.Linear(in_features=hidden_units*56*56, #*13*13,  # =flattened_size
                  out_features=output_shape)
    )

  def forward(self, x: torch.Tensor):
    x = self.conv_block_1(x)
    x = self.conv_block_2(x)
    x = self.classifier(x)
    return x
    # return self.classifier(self.conv_block_2(self.conv_block_1(x))) # <- leverage the benefits of operator fusion

In [ ]:
!ls going_modular/

In [ ]:
!cat going_modular/model_builder.py

In [ ]:
import torch
from torch import nn

class TinyVGG2(nn.Module):
  """Creates the TinyVGG architecture.

  Replicates the TinyVGG architecture from the CNN explainer website in PyTorch.
  See the original architecture here: https://poloclub.github.io/cnn-explainer/

  Args:
    input_shape: An integer indicating number of input channels.
    hidden_units: An integer indicating number of hidden units between layers.
    output_shape: An integer indicating number of output units.
  """
  def __init__(self, input_shape: int, hidden_units: int, output_shape: int) -> None:
    super().__init__()
    self.conv_block_1 = nn.Sequential(
        nn.Conv2d(in_channels=input_shape,
                  out_channels=hidden_units,
                  kernel_size=3,
                  stride=1,
                  padding=1),
        nn.ReLU(),
        nn.Conv2d(in_channels=hidden_units,
                  out_channels=hidden_units,
                  kernel_size=3,
                  stride=1,
                  padding=1),
        nn.ReLU(),
        nn.MaxPool2d(kernel_size=2,
                     stride=2)
    )
    self.conv_block_2 = nn.Sequential(
        nn.Conv2d(hidden_units, hidden_units, kernel_size=3, padding=1),
        nn.ReLU(),
        nn.Conv2d(hidden_units, hidden_units, kernel_size=3, padding=1),
        nn.ReLU(),
        nn.MaxPool2d(2)
    )
    self.classifier = nn.Sequential(
        nn.Flatten(),
        # Where did this in_features shape come from?
        # It's because each layer of our network compresses and changes the shape of our inputs data.
        nn.Linear(in_features=hidden_units*56*56, #*13*13,
                  out_features=output_shape)
    )

  def forward(self, x: torch.Tensor):
    x = self.conv_block_1(x)
    x = self.conv_block_2(x)
    x = self.classifier(x)
    return x

In [ ]:
import torch
device = "cuda" if torch.cuda.is_available() else "cpu"

torch.manual_seed(42)
model_4 = TinyVGG2(input_shape=3,  # number of color channels (3 for RGB)
                   hidden_units=10,
                   output_shape=len(class_names)).to(device)
model_4

In [ ]:
# Now instead of coding the TinyVGG model from scratch every time, we can import it using:

import torch
# Import model_builder.py
from going_modular import model_builder

# Create an instance of TinyVGG - Instantiate an instance of the model from the "model_builder.py" script
torch.manual_seed(42)
model_5 = model_builder.TinyVGG(input_shape=3,  # number of color channels (3 for RGB)
                                hidden_units=20,
                                output_shape=len(class_names)).to(device)
model_5

In [ ]:
# engine.py

%%writefile going_modular/engine.py
"""
Contains functions for training and testing a PyTorch model.
"""
import torch

from tqdm.auto import tqdm
from typing import Dict, List, Tuple

def train_step(model: torch.nn.Module,
               dataloader: torch.utils.data.DataLoader,
               loss_fn: torch.nn.Module,
               optimizer: torch.optim.Optimizer,
               device: torch.device) -> Tuple[float, float]:
  """Trains a PyTorch model for a single epoch.

  Turns a target PyTorch model to training mode and then
  runs through all of the required training steps (forward
  pass, loss calculation, optimizer step).

  Args:
    model: A PyTorch model to be trained.
    dataloader: A DataLoader instance for the model to be trained on.
    loss_fn: A PyTorch loss function to minimize.
    optimizer: A PyTorch optimizer to help minimize the loss function.
    device: A target device to compute on (e.g. "cuda" or "cpu").

  Returns:
    A tuple of training loss and training accuracy metrics.
    In the form (train_loss, train_accuracy). For example:

    (0.1112, 0.8743)
  """
  # Put model in train mode
  model.train()

  # Set up train loss and accuracy values
  train_loss, train_acc = 0, 0

  # Loop through data loader data batches
  for batch, (X, y) in enumerate(dataloader):
    # Send data to target device
    X, y = X.to(device), y.to(device)

    # 1. Forward pass
    y_pred = model(X)

    # 2. Calculate and accumulate loss
    loss = loss_fn(y_pred, y)
    train_loss += loss.item()

    # 3. Optimizer zero grad
    optimizer.zero_grad()

    # 4. Loss backward
    loss.backward()

    # 5. Optimizer step
    optimizer.step()

    # Calculate and accumulate accuracy metric across all batches
    y_pred_class = torch.argmax(torch.softmax(y_pred, dim=1), dim=1)
    train_acc += (y_pred_class == y).sum().item() / len(y_pred)

  # Adjust metrics to get average loss and accuracy per batch
  train_loss /= len(dataloader)
  train_acc /= len(dataloader)
  return train_loss, train_acc


def test_step(model: torch.nn.Module,
              dataloader: torch.utils.data.DataLoader,
              loss_fn: torch.nn.Module,
              device: torch.device) -> Tuple[float, float]:
  """Tests a PyTorch model for a single epoch.

  Turns a target PyTorch model to "eval" mode and then performs
  a forward pass on a testing dataset.

  Args:
    model: A PyTorch model to be tested.
    dataloader: A DataLoader instance for the model to be tested on.
    loss_fn: A PyTorch loss function to calculate loss on the test data.
    device: A target device to compute on (e.g. "cuda" or "cpu").

  Returns:
    A tuple of testing loss and testing accuracy metrics.
    In the form (test_loss, test_accuracy). For example:

    (0.0223, 0.8985)
  """
  # Put model in eval mode
  model.eval()

  # Set up test loss and accuracy values
  test_loss, test_acc = 0, 0

  # Turn on inference context manager
  with torch.inference_mode():
    # Loop through DataLoader batches
    for batch, (X, y) in enumerate(dataloader):
      # Send data to target device
      X, y = X.to(device), y.to(device)

      # 1. Forward pass
      test_pred_logits = model(X)

      # 2. Calculate and accumulate loss
      loss = loss_fn(test_pred_logits, y)
      test_loss += loss.item()

      # Calculate and accumulate accuracy
      test_pred_labels = test_pred_logits.argmax(dim=1)
      test_acc += ((test_pred_labels == y).sum().item() / len(test_pred_labels))

  # Adjust metrics to get average loss and accuracy per batch
  test_loss /= len(dataloader)
  test_acc /= len(dataloader)
  return test_loss, test_acc


def train(model: torch.nn.Module,
          train_dataloader: torch.utils.data.DataLoader,
          test_dataloader: torch.utils.data.DataLoader,
          optimizer: torch.optim.Optimizer,
          loss_fn: torch.nn.Module,
          epochs: int,
          device: torch.device) -> Dict[str, List]:
  """Trains and tests a PyTorch model.

  Sends a target PyTorch model through train_step() and test_step()
  functions for a number of epochs, training and testing the model
  in the same epoch loop.

  Calculates, prints and stores evaluation metrics throughout.

  Args:
    model: A PyTorch model to be trained and tested.
    train_dataloader: A DataLoader instance for the model to be trained on.
    test_dataloader: A DataLoader instance for the model to be tested on.
    optimizer: A PyTorch optimizer to help minimize the loss function.
    loss_fn: A PyTorch loss function to calculate loss on both datasets.
    epochs: An integer indicating how many epochs to train for.
    device: A target device to compute on (e.g. "cuda" or "cpu").

  Returns:
    A dictionary of training and testing loss as well as training and
    testing accuracy metrics. Each metric has a value in a list for
    each epoch.
    In the form: {train_loss: [...],
                  train_acc: [...],
                  test_loss: [...],
                  test_acc: [...]}
    For example if training for epochs=2:
                 {train_loss: [2.0616, 1.0537],
                  train_acc: [0.3945, 0.3945],
                  test_loss: [1.2641, 1.5706],
                  test_acc: [0.3400, 0.2973]}
  """
  # Create empty results dictionary
  results = {"train_loss": [],
      "train_acc": [],
      "test_loss": [],
      "test_acc": []
  }

  # Loop through training and testing steps for a number of epochs
  for epoch in tqdm(range(epochs)):
    train_loss, train_acc = train_step(model=model,
                                       dataloader=train_dataloader,
                                       loss_fn=loss_fn,
                                       optimizer=optimizer,
                                       device=device)
    test_loss, test_acc = test_step(model=model,
                                    dataloader=test_dataloader,
                                    loss_fn=loss_fn,
                                    device=device)

    # Print out what's happening
    print(
        f"Epoch: {epoch+1} | "
        f"train_loss: {train_loss:.4f} | "
        f"train_acc: {train_acc:.4f} | "
        f"test_loss: {test_loss:.4f} | "
        f"test_acc: {test_acc:.4f}"
    )

    # Update results dictionary
    results["train_loss"].append(train_loss)
    results["train_acc"].append(train_acc)
    results["test_loss"].append(test_loss)
    results["test_acc"].append(test_acc)

  # Return the filled results at the end of the epochs
  return results

In [ ]:
from torch import nn
# Set up loss function and optimizer
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(params=model_5.parameters(), lr=0.001)

# Now we've got the engine.py script, we can import functions from it via:

from going_modular import engine

# Use train() by calling it from engine.py
engine.train(model=model_4,
             train_dataloader=train_dataloader,
             test_dataloader=test_dataloader,
             optimizer=optimizer,
             loss_fn=loss_fn,
             epochs=10,
             device=device)

In [ ]:
from going_modular import engine

# Use train() by calling it from engine.py
engine.train(model=model_5,
             train_dataloader=train_dataloader,
             test_dataloader=test_dataloader,
             optimizer=optimizer,
             loss_fn=loss_fn,
             epochs=15,
             device=device)

In [ ]:
# utils.py

%%writefile going_modular/utils.py
"""
Contains various utility functions for PyTorch model training and saving.
"""
import torch
from pathlib import Path

def save_model(model: torch.nn.Module,
               target_dir: str,
               model_name: str):
  """Saves a PyTorch model to a target directory.

  Args:
    model: A target PyTorch model to save.
    target_dir: A directory for saving the model to.
    model_name: A filename for the saved model. Should include
      either ".pth" or ".pt" as the file extension.

  Example usage:
    save_model(model=model_0,
               target_dir="models",
               model_name="05_going_modular_tingvgg_model.pth")
  """
  # Create a target directory
  target_dir_path = Path(target_dir)
  target_dir_path.mkdir(parents=True, exist_ok=True)

  # Create model save path
  assert model_name.endswith(".pth") or model_name.endswith(".pt"), "model_name should end with '.pt' or '.pth'"  # text to display if assert check fails
  model_save_path = target_dir / model_name

  # Save the model state_dict()
  print(f"[INFO] Saving model to: {model_save_path}")
  torch.save(obj=model.state_dict(),
             f=model_save_path)

In [ ]:
!ls going_modular/


In [ ]:
target_dir = Path("saved_models")
model_name = "model_5.pth"

# Now if we wanted to use our save_model() function we can import it and use it via:

from going_modular import utils

# Save a model to file
utils.save_model(model=model_5,
                 target_dir=target_dir,
                 model_name=model_name)

In [ ]:
# Save a model to file
utils.save_model(model=model_4,
                 target_dir=target_dir,
                 model_name="model_4.pth")

In [ ]:
# train0.py

%%writefile going_modular/train0.py
"""
Trains a PyTorch image classification model using device-agnostic code.
"""
import os
import torch
import data_setup, engine, model_builder, utils

from torchvision import transforms
from pathlib import Path

# Set up hyperparameters
NUM_EPOCHS = 5
BATCH_SIZE = 32
HIDDEN_UNITS = 10
LEARNING_RATE = 0.001

# Set up directories
train_dir = "data/pizza_steak_sushi/train"
test_dir = "data/pizza_steak_sushi/test"

# Set up target device
device = "cuda" if torch.cuda.is_available() else "cpu"

# Create transforms
data_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor()
])

# Create DataLoaders with help from data_setup.py
train_dataloader, test_dataloader, class_names = data_setup.create_dataloaders(
    train_dir=train_dir,
    test_dir=test_dir,
    transform=data_transform,
    batch_size=BATCH_SIZE
)

# Create model with help from model_builder.py
model = model_builder.TinyVGG(
    input_shape=3,
    hidden_units=HIDDEN_UNITS,
    output_shape=len(class_names)
).to(device)

# Set loss and optimizer
loss_fn = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(),
                            lr=LEARNING_RATE)
#optimizer = torch.optim.Adam(model.parameters(),
#                             lr=LEARNING_RATE)

# Start training with help from engine.py
engine.train(model=model,
             train_dataloader=train_dataloader,
             test_dataloader=test_dataloader,
             loss_fn=loss_fn,
             optimizer=optimizer,
             epochs=NUM_EPOCHS,
             device=device)

# Save the model with help from utils.py
utils.save_model(
    model=model,
    target_dir=Path("models"),
    model_name="05_going_modular_script_mode_tinyvgg_model.pth"
)

In [ ]:
# Now we can train a PyTorch model by running the following line on the command line:
!python going_modular/train0.py

In [ ]:
!python going_modular/get_data.py

In [ ]:
!ls data/pizza_steak_sushi/test/pizza

In [ ]:
# train.py with argparse

%%writefile train.py
"""
Trains a PyTorch image classification model using device-agnostic code.
"""
import os
import torch
import argparse

from going_modular import data_setup, engine, model_builder, utils
from torchvision import transforms
from pathlib import Path

# Create a parser
parser = argparse.ArgumentParser(description="Get some hyperparameters.")

# Create an arg for number of epochs
parser.add_argument("--num_epochs",
                    default=10,
                    type=int,
                    help="the number of epochs to train for")

# Create an arg for batch size
parser.add_argument("--batch_size",
                    default=32,
                    type=int,
                    help="number of samples per batch")

# Create an arg for hidden units
parser.add_argument("--hidden_units",
                    default=10,
                    type=int,
                    help="number of hidden units in hidden layers")

# Create an arg for learning rate
parser.add_argument("--learning_rate",
                    default=0.001,
                    type=float,
                    help="learning rate to use for model")

# Create an arg for training directory
parser.add_argument("--train_dir",
                    default="data/pizza_steak_sushi/train",
                    type=str,
                    help="directory file path to training data in standard image classification format")

# Create an arg for test directory
parser.add_argument("--test_dir",
                    default="data/pizza_steak_sushi/test",
                    type=str,
                    help="directory file path to testing data in standard image classification format")

# Create an arg for model name
parser.add_argument("--model_name",
                    default="model_0.pth",
                    type=str,
                    help="model name to save")

# Get our arguments from the parser
args = parser.parse_args()

# Set up hyperparameters
NUM_EPOCHS = args.num_epochs
BATCH_SIZE = args.batch_size
HIDDEN_UNITS = args.hidden_units
LEARNING_RATE = args.learning_rate
print(f"[INFO] Training a model for {NUM_EPOCHS} epochs with batch size {BATCH_SIZE}, hidden units {HIDDEN_UNITS} and learning rate {LEARNING_RATE}...")

# Set up directories
train_dir = args.train_dir
test_dir = args.test_dir
print(f"[INFO] Training data file: {train_dir}")
print(f"[INFO] Testing data file: {test_dir}")

model_name = args.model_name
print(f"[INFO] Model will be saved as: {model_name}")

# Set up target device
device = "cuda" if torch.cuda.is_available() else "cpu"

# Create transforms
data_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.ToTensor()
])

# Create DataLoaders with help from data_setup.py
train_dataloader, test_dataloader, class_names = data_setup.create_dataloaders(
    train_dir=train_dir,
    test_dir=test_dir,
    transform=data_transform,
    batch_size=BATCH_SIZE
)

# Create model with help from model_builder.py
model = model_builder.TinyVGG(
    input_shape=3,
    hidden_units=HIDDEN_UNITS,
    output_shape=len(class_names)
).to(device)

# Set loss and optimizer
loss_fn = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(),
                            lr=LEARNING_RATE)
#optimizer = torch.optim.Adam(model.parameters(),
#                             lr=LEARNING_RATE)

# Start training with help from engine.py
engine.train(model=model,
             train_dataloader=train_dataloader,
             test_dataloader=test_dataloader,
             loss_fn=loss_fn,
             optimizer=optimizer,
             epochs=NUM_EPOCHS,
             device=device)

# Save the model with help from utils.py
target_dir = Path("saved_models")

utils.save_model(
    model=model,
    target_dir=target_dir,
    model_name=model_name
)

In [ ]:
!python train.py --num_epochs 15 --batch_size 32 --hidden_units 20 --model_name model_1.pth

In [ ]:
# Takes too long, too slow with batch_size=64 or 128 and hidden_units=64 or 128
!python train.py --num_epochs 3 --batch_size 64 --hidden_units 16 --learning_rate 0.002 --model_name model_2.pth

In [ ]:
!ls models
!ls saved_models/

In [ ]:
%%writefile predict.py
"""
Makes predictions with a trained PyTorch model and saves the results to file.
"""
import torch
import torchvision
import argparse
import matplotlib.pyplot as plt

from going_modular import model_builder
from torchvision import transforms

# Create a parser
parser = argparse.ArgumentParser()

# Create an arg for model path
parser.add_argument("--model_path",
                    default="models/05_going_modular_script_mode_tinyvgg_model.pth",
                    type=str,
                    help="filepath of model to use for prediction")

# Create an arg for image path
parser.add_argument("--image_path",
                    default="data/pizza_steak_sushi/train/pizza/12301.jpg",
                    type=str,
                    help="filepath of image to predict on")

# Create an arg for transform type
parser.add_argument("--transform",
                    default="no",
                    type=str,
                    help="yes to transform using horizontal flip, no is default")

# Create an arg for hidden units
parser.add_argument("--hidden_units",
                    default=10,
                    type=int,
                    help="number of hidden units used by model")

args = parser.parse_args()

model_path = args.model_path
image_path = args.image_path
transform = args.transform
hidden_units = args.hidden_units

print(f"[INFO] Predicting on {image_path} with {model_path}")

# Set up class names
class_names = ["pizza", "steak", "sushi"]

# Set up device
device = "cuda" if torch.cuda.is_available() else "cpu"

# Need to use same hyperparameters as saved model
model = model_builder.TinyVGG(input_shape=3,
                              hidden_units=hidden_units,
                              output_shape=3).to(device)  # len(class_names) = 3

# Load in the saved model state dictionary from file
model.load_state_dict(torch.load(model_path))

# 1. Load in an image and convert tensor values to float32
target_image = torchvision.io.read_image(str(image_path)).type(torch.float32)

# 2. Divide the image pixel values by 255 to get them between 0 and 1
target_image /= 255

# 3. Transform if necessary
if transform == "yes":
  data_transform_flip = transforms.Compose([
    transforms.Resize((224,224)),#64,64)),
    # Flip images randomly on the horizontal
    transforms.RandomHorizontalFlip(p=0.5),  # p = probability of flip, 0.5 = 50% chance
    #transforms.ToTensor()
  ])
  target_image = data_transform_flip(target_image)
  print("Using data_transform_flip")
else:  # Resize the image to be the same size as the model
  data_transform = transforms.Compose([
    transforms.Resize((224,224)),#64,64)),
    #transforms.ToTensor()
  ])
  target_image = data_transform(target_image)

# 4. Make sure model is on target device
model.to(device)

# 5. Turn on model evaluation and inference modes
model.eval()
with torch.inference_mode():

  # Add an extra dimension to image
  target_image = target_image.unsqueeze(dim=0)
  # Make a prediction on image with an extra dimension and send it to the target device
  target_image_pred = model(target_image.to(device))

# 6. Convert logits to probabilities
target_image_pred_probs = torch.softmax(target_image_pred, dim=1)

# 7. Convert probs to label
target_image_pred_label = torch.argmax(target_image_pred_probs, dim=1)

# 8. Plot the image alongside the prediction and prediction probability
plt.imshow(target_image.squeeze().permute(1, 2, 0))  # make sure it's right size for matplotlib
if class_names:
  title = f"Pred: {class_names[target_image_pred_label.cpu()]} | Prob: {target_image_pred_probs.max().cpu():.3f}"
else:
  title = f"Pred: {target_image_pred_label} | Prob: {target_image_pred_probs.max().cpu():.3f}"
plt.title(title)
plt.axis(False);

print(f"[INFO] Prediction label: {class_names[target_image_pred_label]}, prediction probability: {target_image_pred_probs.max():.3f}")

In [ ]:
!ls saved_models/

In [ ]:
!python predict.py --image_path data/pizza_steak_sushi/train/pizza/300869.jpg --model_path saved_models/model_5.pth --hidden_units 20 --transform yes
!python predict.py --image_path data/pizza_steak_sushi/train/pizza/300869.jpg --model_path saved_models/model_5.pth --hidden_units 20

In [ ]:
!python predict.py --image_path data/pizza_steak_sushi/test/pizza/61656.jpg --model_path saved_models/model_5.pth --hidden_units 20
!python predict.py --image_path data/pizza_steak_sushi/test/pizza/61656.jpg --model_path saved_models/model_5.pth --hidden_units 20 --transform yes

In [ ]:
# get_custom_data.py

%%writefile going_modular/get_custom_data.py
"""
Contains functionality to download custom images from GitHub
"""
import requests
from pathlib import Path

data_path = Path("data")

# Get multiple custom images
""" to get raw address:  right click jpeg file name in main repository view, select copy link address, paste into browser and enter
    then right click on image, select copy link address = https://github.com/lanehale/pytorch-deep-learning/blob/main/cheese-pizza.jpeg?raw=true
    paste into browser and enter to get url format below
"""
urls = [
    "https://raw.githubusercontent.com/lanehale/pytorch-deep-learning/refs/heads/main/custom_images/cheese-pizza.jpeg",
    "https://raw.githubusercontent.com/lanehale/pytorch-deep-learning/refs/heads/main/custom_images/pizza-slice.jpeg",
    "https://raw.githubusercontent.com/lanehale/pytorch-deep-learning/refs/heads/main/custom_images/pizza-slice2.jpeg",
    "https://raw.githubusercontent.com/lanehale/pytorch-deep-learning/refs/heads/main/custom_images/pizza-sliced.jpeg",
    "https://raw.githubusercontent.com/lanehale/pytorch-deep-learning/refs/heads/main/custom_images/pizza-sliced2.jpeg",
    "https://raw.githubusercontent.com/lanehale/pytorch-deep-learning/refs/heads/main/custom_images/pizza-partial-view.jpeg",
    "https://raw.githubusercontent.com/lanehale/pytorch-deep-learning/refs/heads/main/custom_images/pizza-partial-view2.jpeg",
    "https://raw.githubusercontent.com/lanehale/pytorch-deep-learning/refs/heads/main/custom_images/pizza-side-view.jpeg"
]

filenames = [
    "cheese-pizza.jpeg",
    "pizza-slice.jpeg",
    "pizza-slice2.jpeg",
    "pizza-sliced.jpeg",
    "pizza-sliced2.jpeg",
    "pizza-partial-view.jpeg",
    "pizza-partial-view2.jpeg",
    "pizza-side-view.jpeg"
]

if len(urls) != len(filenames):
  raise ValueError("The number of URLs and filenames must be the same.")

# Download the images if they don't already exist
if (data_path / "cheese-pizza.jpeg").is_file():
  print(f"Custom images already exist, skipping download.")
else:
  for i, url in enumerate(urls):
    try:
      response = requests.get(url)
      response.raise_for_status()  # Raise an exception for HTTP errors

      custom_image_path = data_path / filenames[i]

      with open(custom_image_path, "wb") as f:
        f.write(response.content)

      print(f"DownLoading {custom_image_path}...")

    except requests.exceptions.RequestException as e:
      print(f"Error downloading {url}: {e}")
    except Exception as e:
      print(f"An unexpected error occurred: {e}")

In [ ]:
# Get custom images
!python going_modular/get_custom_data.py

In [ ]:
!ls data

In [ ]:
!python predict.py --image_path data/cheese-pizza.jpeg --model_path saved_models/model_5.pth --hidden_units 20

In [ ]:
!python predict.py --image_path data/pizza-sliced.jpeg --model_path saved_models/model_5.pth --hidden_units 20

In [ ]:
!python predict.py --image_path data/pizza-partial-view.jpeg --model_path saved_models/model_5.pth --hidden_units 20